## A. 
Data exploration
Explore the three datasets. Use your judgment as a human being to describe
what makes you able to tell spam apart from ham. Also explore the differences between the two classes of ham (easy and hard ham). What makes them different? Write approximately one paragraph.

1. ##### Difference between Ham and Spam
    Spam contains a lot of clickbaity titles with links that looks more than suspicious. Very sensational language and the senders adress looks very auto-generated for example: sabrina@mx3.1premio.com.
    Ham on the other hand is more legit and the emails looks like they are sent from either real people or real companies with a newsletter and you get the choice to unsubscribe or not.

2. ##### Difference between Hard and Easy Ham
    Easy ham seems to be mostly conversations between real life people, not alot of code, mostly natural speaking language
    Hard Ham contains more code, and contains mostly of commercials which makes it hard to judge if it's actually wanted by the reciever or not.

## B.
Perform an appropriate train-test split on the each of the datasets. We will use
the training sets to train a classifier, and evaluate its performance against the
test sets.

In [1]:
import pandas as pd
import glob, os

# Read txt file from the folders
easy_ham_folder_path = os.getcwd() + r"/Lab 3/20021010_easy_ham"
hard_ham_folder_path = os.getcwd() + r"Lab 3/20021010_hard_ham"
spam_folder_path = os.getcwd() + r"Lab 3/20021010_spam"

# Read the files from the folders
easy_ham_files = glob.glob(easy_ham_folder_path + "/*.txt")
hard_ham_files = glob.glob(hard_ham_folder_path + "/*.txt")
spam_files = glob.glob(spam_folder_path + "/*.txt")

# Make dataframes
for file in easy_ham_files:
    with open(file, 'r') as f:
        easy_ham_df = pd.DataFrame(f.readlines())

print(easy_ham_df)

Empty DataFrame
Columns: []
Index: []
